In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder


user_encoder = LabelEncoder()
guide_encoder = LabelEncoder()

df_interactions['user_numeric'] = user_encoder.fit_transform(df_interactions['user_id'])
df_interactions['guide_numeric'] = guide_encoder.fit_transform(df_interactions['guide_id'])


interaction_matrix_svd = df_interactions.pivot_table(
    index='user_numeric',
    columns='guide_numeric',
    values='interest_match_score',
    fill_value=0
)


svd = TruncatedSVD(n_components=15, random_state=42)
svd_matrix = svd.fit_transform(interaction_matrix_svd)


reconstructed_matrix = np.dot(svd_matrix, svd.components_)


sample_user_index = 0
user_id = user_encoder.inverse_transform([sample_user_index])[0]


user_predictions = reconstructed_matrix[sample_user_index]
top_guide_indices = user_predictions.argsort()[::-1]


interacted_guides = df_interactions[df_interactions['user_id'] == user_id]['guide_id'].values
interacted_guide_indices = guide_encoder.transform(interacted_guides)
top_guide_indices = [i for i in top_guide_indices if i not in interacted_guide_indices]


top_5_indices = top_guide_indices[:5]
recommended_guide_ids = guide_encoder.inverse_transform(top_5_indices)
predicted_scores = user_predictions[top_5_indices]


recommendations_df = pd.DataFrame({
    "guide_id": recommended_guide_ids,
    "predicted_score": predicted_scores
})


user_row = df_users[df_users['id'] == user_id].iloc[0]
user_interests = "، ".join(df_interests[df_interests['user_id'] == user_id]['interest'].tolist())


recommendations_df = recommendations_df.merge(df_guides, left_on="guide_id", right_on="id")
guide_tags = df_tags.groupby("guide_id")["tag"].apply(lambda x: "، ".join(x)).reset_index()
recommendations_df = recommendations_df.merge(guide_tags, on="guide_id", how="left")


recommendations_df_final = recommendations_df[[
    "name", "tag", "region", "predicted_score"
]].copy()

recommendations_df_final.insert(0, "user_name", user_row["name"])
recommendations_df_final.insert(1, "user_interests", user_interests)
recommendations_df_final.insert(2, "user_region", user_row["region"])

recommendations_df_final = recommendations_df_final.rename(columns={
    "user_name": "المستخدم",
    "user_interests": "اهتماماته",
    "user_region": "منطقته",
    "name": "المرشد المقترح",
    "tag": "اهتمامات المرشد",
    "region": "منطقته (المرشد)",
    "predicted_score": "درجة التوقع"
})


display(recommendations_df_final)

